In [1]:
%reload_kedro

2023-05-09 11:53:20,331 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: /Users/gpalazzo/Desktop/dev/crypto_msc_thesis
2023-05-09 11:53:20,980 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2023-05-09 11:53:21,372 - kedro.extras.extensions.ipython - INFO - ** Kedro project crypto_thesis
2023-05-09 11:53:21,372 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
from crypto_thesis.data_domains.spine import spine_build_target_labels, spine_balance_classes

### Label balancing

In [4]:
df_spine_preproc = catalog.load("spine_preprocessing")
df_spine_logret = catalog.load("spine_log_ret")
spine_params = catalog.load("params:spine_labeling")
spine_cls_bds_params = catalog.load("params:spine_class_bounds")

2023-05-09 11:54:02,478 - kedro.io.data_catalog - INFO - Loading data from `spine_preprocessing` (ParquetDataSet)...
2023-05-09 11:54:02,704 - kedro.io.data_catalog - INFO - Loading data from `spine_log_ret` (ParquetDataSet)...
2023-05-09 11:54:02,722 - kedro.io.data_catalog - INFO - Loading data from `params:spine_labeling` (MemoryDataSet)...
2023-05-09 11:54:02,723 - kedro.io.data_catalog - INFO - Loading data from `params:spine_class_bounds` (MemoryDataSet)...


In [5]:
# test a different tau value to evaluate class balancing
spine_params

{'tau': 0.35}

In [6]:
df_spine_lbl = spine_build_target_labels(df=df_spine_preproc, 
                                         df_log_ret=df_spine_logret, 
                                         label_params=spine_params)

In [7]:
df_spine_lbl.label.value_counts() / df_spine_lbl.shape[0]

bottom    0.630487
top       0.369513
Name: label, dtype: float64

In [8]:
df = spine_balance_classes(df=df_spine_lbl, 
                          class_bounds=spine_cls_bds_params)

2023-05-09 11:54:11,882 - crypto_thesis.data_domains.spine.build_labels - INFO - Checking for class balance
2023-05-09 11:54:11,901 - crypto_thesis.data_domains.spine.build_labels - INFO - Checking spine quality


In [9]:
df.label.value_counts() / df.shape[0]

bottom    0.5
top       0.5
Name: label, dtype: float64

### Volume bar time interval plots

In [ ]:
df_spine = catalog.load("spine_labeled_balanced")
df_ftes = catalog.load("fte_binance").dropna()

In [ ]:
df_spine_aux = df_spine.copy()

df_spine_aux = df_spine_aux.merge(df_ftes[["open_time", "close_time"]], \
                                  on=["open_time", "close_time"] , \
                                 how="inner")

df_spine_aux.loc[:, "open_close_duration_sec"] = (df_spine_aux["close_time"] - df_spine_aux["open_time"]).dt.total_seconds()
df_spine_aux.loc[:, "close_target_duration_sec"] = (df_spine_aux["target_time"] - df_spine_aux["close_time"]).dt.total_seconds()

df_spine_aux.loc[:, "open_close_duration_min"] = df_spine_aux["open_close_duration_sec"] / 60.0
df_spine_aux.loc[:, "close_target_duration_min"] = df_spine_aux["close_target_duration_sec"] / 60.0

In [ ]:
df_spine_aux.head()

In [ ]:
df_plot = df_spine_aux[["close_time", "open_close_duration_min", "close_target_duration_min"]] \
                        .set_index("close_time") \
                        .sort_index()

plt.figure(figsize=(12, 5))

# plot = sns.boxplot(data=df_plot, palette="gray")
plot = sns.boxplot(data=df_plot)

fig = plot.get_figure()
fig.savefig("/Users/gpalazzo/Desktop/fig_boxplot_time_windows.png")

In [ ]:
df_plot.describe()